# Creating File to check Recall of Index

In [1]:
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize

import xapian
import string
import sys
import time
import glob
from pandas import DataFrame

import spacy
from spacy import displacy
from collections import Counter

import re, math

In [2]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [3]:
import swifter

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
devset_file_path = "devset.json"
devset = pd.read_json(open(devset_file_path),orient='index')

In [5]:
len(devset) #5001 lenght of dev set

5001

In [40]:
start = time.time()
# Open the database for searching.
database = xapian.Database("db-index/")

# Start an enquire session.
enquire = xapian.Enquire(database)

In [42]:
def get_topk_evidence(claim):
    
    
    query_string = claim
    #print ("Claim:" , query_string )
    # Parse the query string to produce a Xapian::Query object.
    qp = xapian.QueryParser()
    stemmer = xapian.Stem("english")
    qp.set_stemmer(stemmer)
    
    qp.set_database(database)
    qp.set_stemming_strategy(xapian.QueryParser.STEM_SOME)
    query = qp.parse_query(query_string)
    #print (query)

    # Find the top 10 results for the query.
    enquire.set_query(query)
    matches = enquire.get_mset(0,5)

    # Display the results.
    #print ("%i results found." % matches.get_matches_estimated())
    #print ("Results 1-%i:" % matches.size(),'\n')

    doc=[]
    for m in matches: 
        a = m.document.get_data().decode('utf-8')
        #print(a, '\n')
        a= a.split(' ', 2)[0:2]
        if a[1].isnumeric():
            a[1] = int(a[1])
            doc.append(a)
        
    return doc
    

In [45]:
devset['evidence'] = devset['claim'].swifter.apply(lambda x: get_topk_evidence(x))

In [46]:
devset.to_json('check_recall_file.json', orient='index') 